# Playing atari games like a pro

Ever got frustaded for your lack of coordination when it comes to pressing button and breathing at the same time?

(image of discordinated human) YES!!!!

So this article is for you! With the help of deep learning lets help our laziness to make the computer play a game for us while we focus on breathing watching bogo cat play its songs.

(image of bongo cat)

## Requirement for this article

This time we will not go through all the basics of neuro networks. So make sure that you know at least some basics of deep learning, convolution layers and quantum physics.

Also we will be using **pytorch** for runing and trainig our models. So knowing a bit about **pytorch** also helps.

## Deep reinforcement learning?

You probably head about the most common types of machine learning: supervised and unsupervised. There is a third category of learning named reinforcement learning. So how does these three brothers differs from each other?

#### Supervised learning

This is the most common kind of machine learning. In this type the model has access to both the question (input) and the answer (output). It then can make a guess about the input and calculate how much it missed from the real output. Then use this difference to update its weights.

This type of learning is really good too use in object recognition.

(image of classifier)

#### Unsupervised learning

In this case the model has no access to the answers to the question. E.g it only has the input data but not the output. So the machine will try to model the input by understanding the differences and similarities between them. 

This type of learning can be used for clustering or even audio/video synthesis.

(image of GAN)

#### Reinforcement learning

Now this less used type of learning has some peculiarities. First it can interact with the environment by a set of actions and then observes how its actions changes the environment (also called state).

This type of AI has the objective to maximize the rewards it gets from the environment. Using games as an exemple, the score can be a reward, so the machine will try to learn what kind of actions are the best to use in each possible state so that it can achieve the maximum amount of rewards.

Like in this article, this type of learning can be used to play games. But on a more useful note, it can be used to teach robots to move on a most efficient way or even teach self-driving cars to drive as safe as possible.

(image of robots high fiveing each other)

### Where the "deep" comes from

Like with deep neural networks for supervised learning, when we say deep reinforcement learning, what we mean is that we will use a deep neural network to model what are the best actions to choose depending on the state the evironment is now. 

...and it sounds cooler.


## OpenAI gym

OpenAI is a non-profit AI research company that provide us with a bunch of cool tool to train reiforcement leaning models. These are normaly small games with an easy interface for rendering and action input. 
We could use an emulator to be able to run any game we can want, however in this article we want to focus on the model training part so we will be using the OpenAI gym libraries to create and interact with our games.

Installing the library is pretty straighforward. Just run **pip install gym** or **pip install --user gym** to keep the library local, and if you are using Gentoo, you need to add that **--user**

Here lets try to import and play our game with random actions for now.
